## Tutorial 2

This tutorial contains the following demos:
1. Monte Carlo estimation
1. Non-linear features: good and bad
1. Model selection: cross-validation
1. Standardization and regularization

In [ ]:
## Code source: Xianyao Zhang
## with helper functions based on Sebastian Curi and Andreas Krause, and Jaques Grobler (sklearn demos).
# License: BSD 3 clause

import sys
sys.path.append('./helper_files')  # helper files provided on the course website

import numpy as np
from typing import Callable

np.set_printoptions(precision=4, suppress=True, linewidth=100)
np.random.seed(42)

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10., 5.]
plt.rcParams['figure.dpi'] = 100


In [ ]:
def plot_function(fn: Callable, a, b, *, n_points=10000, linewidth=1, label='fn', ax=None):
    x = np.linspace(a, b, n_points)
    y = fn(x)
    if ax is None:
        ax = plt.gca()
    ax.plot(x, y, linewidth=linewidth, label=label)

### Monte Carlo integration

In [ ]:
from scipy.integrate import quadrature
from scipy import stats

fn = lambda x: np.exp(np.sin(x**3))
a, b = -1, 1

plot_function(fn, a, b)
plot_function(stats.uniform(loc=a, scale=b-a).pdf, a, b, label='uniform')
plot_function(stats.truncnorm(a, b).pdf, a, b, label='truncnorm')
def truncexpon_pdf(a, b):
    def fn(x):
        return stats.truncexpon(b-a).pdf(x-a)
    return fn
plot_function(truncexpon_pdf(a, b), a, b, label='truncexpon')

plt.legend(); plt.show()

quad_integral, quad_error = quadrature(fn, a=a, b=b)
print(f'quadrature: int={quad_integral: .4f}, err={quad_error:.4e}')

def monte_carlo_integrate(fn: Callable, a, b, *, sample_distribution: str='uniform', n_samples=1000):
    if sample_distribution == 'uniform':
        rv = stats.uniform(loc=a, scale=b-a)
        samples = rv.rvs(size=n_samples)
        pdf = rv.pdf(samples)
    elif sample_distribution == 'truncnorm':
        rv = stats.truncnorm(a, b)
        samples = rv.rvs(size=n_samples)
        pdf = rv.pdf(samples)
    elif sample_distribution == 'truncexpon':
        rv = stats.truncexpon(b-a)
        samples = rv.rvs(size=n_samples) + a
        pdf = rv.pdf(samples-a)
    else:
        raise ValueError(f'Unsupported distribution: {sample_distribution}')
    
    fn_values = fn(samples)
    mc_samples = fn_values / pdf
    mc_estimate = np.mean(mc_samples)
    mc_std = np.std(mc_samples, ddof=1)
    return mc_estimate, mc_std

mc_integral, mc_error = monte_carlo_integrate(fn=fn, a=a, b=b, n_samples=100000, sample_distribution='uniform')
print(f'uniform_mc: int={mc_integral: .4f}, err={mc_error: .4e}')
mc_integral, mc_error = monte_carlo_integrate(fn=fn, a=a, b=b, n_samples=100000, sample_distribution='truncexpon')
print(f'truncexpon_mc: int={mc_integral: .4f}, err={mc_error: .4e}')
mc_integral, mc_error = monte_carlo_integrate(fn=fn, a=a, b=b, n_samples=100000, sample_distribution='truncnorm')
print(f'truncnormal_mc: int={mc_integral: .4f}, err={mc_error: .4e}')

### Non-linear features: good and bad

In [ ]:
from collections import OrderedDict
np.random.seed(42)

true_function = lambda x: np.sin(x) + 2*np.sin(x)*np.cos(x) + .5*np.exp(-(x)**2+1)
# true_function = lambda x: 5*x + 3*x**2 - 0.5*x**4
# true_function = lambda x: 5*x + 3*x**2 - 0.05*x**4 + 15*np.exp(-x**2+1)

num_points = 50
a, b = -8, 8
x = np.random.uniform(a, b, size=(num_points,))
PLOT_FN_X = np.linspace(a, b, 5000)

true_y = true_function(x)
noisy_y = true_y + np.random.normal(scale=0.1, size=x.shape)
plt.scatter(x, true_y, s=3, label='true')
plt.scatter(x, noisy_y, s=3, label='noisy')
plot_function(true_function, a, b, label='true_fn')
plt.legend()
plt.show()

def make_poly(k):
    return lambda x: np.power(x, k)

nonlinears = {
    'const': lambda x: np.ones_like(x),
    **{f'poly_{k}': make_poly(k) for k in range(1, 8)}, 
    'sinx': lambda x: np.sin(x),
    'cosx': lambda x: np.cos(x),
    'sin2x': lambda x: np.sin(2*x),
    'cos2x': lambda x: np.cos(2*x),
    'sin3x': lambda x: np.sin(3*x),
    'cos3x': lambda x: np.cos(3*x),
#     'expx': lambda x: np.exp(x),
#     'expx2': lambda x: np.exp(x**2),
    'exp-x2': lambda x: np.exp(-x**2),
    'logx': lambda x: np.log(x),
}

nonlinears = OrderedDict(sorted(nonlinears.items(), key=lambda t: t[0]))

In [ ]:
def get_nonlinear_function(functions: OrderedDict, weights):
    # f(x) = w[0]f[0](x) + w[1]f[1](x) + ... 
    assert len(functions) == len(weights)
    def ret_fn(x):
        ret = np.zeros_like(x)
        for i, (name, fn) in enumerate(functions.items()):
            ret += weights[i] * fn(x)
        return ret
    return ret_fn

def get_X_functions(x, fns: OrderedDict):
    X = np.concatenate([f(x)[:, np.newaxis] for k, f in fns.items()], axis=1)
    return X

In [ ]:
fns_poly = OrderedDict([(k, v) for k, v in nonlinears.items() if k.startswith('poly') or k == 'const'])
fns_sin_exp = OrderedDict([(k, v) for k, v in nonlinears.items() if k.startswith('sin') or k.startswith('exp') or k == 'const'])

In [ ]:
def get_X_poly(x):
    return get_X_functions(x, fns_poly)

def get_X_sin_exp(x):
    return get_X_functions(x, fns_sin_exp)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def plot_weighted_fns(fns, weights, a, b, label='fn'):
    fn = get_nonlinear_function(fns, weights)
    plot_function(fn, a, b, label=label)
    return

def fit_nonlinear_and_show(reg, x, y, fns: OrderedDict, name=None):
    X = get_X_functions(x, fns)

    reg.fit(X, y)
    pred_y = reg.predict(X)
    
    print('coeffs=', reg.coef_)
    print(f'score={reg.score(X, y):.4f}')
    print(f'mse={mean_squared_error(y, pred_y):.4f}')
    plt.scatter(x, pred_y, s=3, label=name or 'pred')
    plt.scatter(x, y, s=3, label='true')
    return reg

In [ ]:
ax = plt.subplot(2,2,1)
reg1 = fit_nonlinear_and_show(LinearRegression(fit_intercept=False), x, noisy_y, fns_poly, name='poly')
plot_weighted_fns(fns_poly, reg1.coef_, a, b)
plt.legend()

ax = plt.subplot(2,2,2)
reg2 = fit_nonlinear_and_show(LinearRegression(fit_intercept=False), x, noisy_y, fns_sin_exp, name='sin_exp')
plot_weighted_fns(fns_sin_exp, reg2.coef_, a, b)
plt.legend()

ax = plt.subplot(2,2,3)
reg3 = fit_nonlinear_and_show(LinearRegression(fit_intercept=False), x, true_y, fns_poly, name='poly')
plot_weighted_fns(fns_poly, reg3.coef_, a, b)
plt.legend()

ax = plt.subplot(2,2,4)
reg4 = fit_nonlinear_and_show(LinearRegression(fit_intercept=False), x, true_y, fns_sin_exp, name='sin_exp')
plot_weighted_fns(fns_sin_exp, reg4.coef_, a, b)
plt.legend()

plt.show()

In [ ]:
X_sin_exp = get_X_sin_exp(x)

In [ ]:
fns_all = OrderedDict(list(fns_poly.items()) + list(fns_sin_exp.items()))
reg = fit_nonlinear_and_show(LinearRegression(fit_intercept=False), x, noisy_y, fns_all, name='all')
plot_weighted_fns(fns_all, reg.coef_, a, b)
plt.legend()
plt.show()

### Model selection: cross-validation

In [ ]:
from sklearn.model_selection import KFold
n_folds = 5

# Generate a testing set 

def my_cross_val(reg, fns, x, y, *, cv, a, b):
    X = get_X_functions(x, fns)
    plot_x = x
    n_points = X.shape[0]
    kf = KFold(n_splits=cv)

#     indices = [list(range(k*n_points//cv, (k+1)*n_points//cv)) for k in range(cv-1)]
#     indices.append(list(range((cv-1)*n_points//cv, n_points)))

    cv_score = []
    for i, (train_indices, val_indices) in enumerate(kf.split(X)):
        print(f'CV fold {i}')
#         train_indices = np.setdiff1d(np.arange(n_points), val_ind)
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]
        print(X_train.shape, X_val.shape)
        plot_x_train = plot_x[train_indices]
        plot_x_val = plot_x[val_indices]
        
        reg.fit(X_train, y_train)
        y_val_pred = reg.predict(X_val)
        y_train_pred = reg.predict(X_train)
        cv_score.append(-mean_squared_error(y_val_pred, y_val))
        
        plt.figure()
        print(reg.coef_)
        plt.scatter(plot_x_train, y_train_pred, s=3, label=f'fold_{i}_train')
        plt.scatter(plot_x_train, y_train, s=3, label=f'fold_{i}_trtrue')
        plt.scatter(plot_x_val, y_val_pred, s=16, marker='x', label=f'fold_{i}_val')
        plt.scatter(plot_x_val, y_val, s=16, marker='*', label=f'fold_{i}_val_true')
        plot_weighted_fns(fns, reg.coef_, a, b)
        plt.legend()
        plt.show()
    return cv_score
        
my_cross_val(LinearRegression(normalize=True, fit_intercept=False), fns_sin_exp, x, noisy_y, cv=n_folds, a=a, b=b)

In [ ]:
my_cross_val(LinearRegression(normalize=True, fit_intercept=False), fns_poly, x, noisy_y, cv=n_folds, a=a, b=b)

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate, cross_val_predict

reg = LinearRegression(normalize=False)
scores_poly = cross_val_score(reg, get_X_poly(x), noisy_y, cv=n_folds, scoring='neg_mean_squared_error')
print('scores_poly =', scores_poly)
print(f'avg = {np.array(scores_poly).mean():.4f}')

In [ ]:
reg = LinearRegression(normalize=False)
scores_sin_exp = cross_val_score(reg, get_X_sin_exp(x), noisy_y, cv=n_folds, scoring='neg_mean_squared_error')
print('scores_sin_exp =', scores_sin_exp)
print(f'avg = {np.array(scores_sin_exp).mean():.4f}')

### Standardization and regularization

In [ ]:
from sklearn import datasets, linear_model
from sklearn.datasets import make_regression
from sklearn.preprocessing import scale, PolynomialFeatures

# Load the diabetes dataset
diabetes = datasets.load_diabetes()

Y_all = diabetes.target
X_std = scale(diabetes.data) # zero mean unit variance
n_all = Y_all.size
n_test = 20

In [ ]:
diabetes.data.mean(axis=0)

In [ ]:
diabetes.data.std(axis=0)

In [ ]:
X_std.std(axis=0)

In [ ]:
diabetes.data.shape

In [ ]:
random_scale = 2**np.random.uniform(-6, 6, size=(1,10))
random_mean = np.random.uniform(-50, 50, size=(1,10))
print(random_scale)
print(random_mean)

In [ ]:
X = X_std*random_scale + random_mean  # X_std = (X-mean(X))/std(X)
print('shape=', X.shape)
print('feature_mean=', X.mean(axis=0))
print('feature_std=', X.std(axis=0))

In [ ]:
reg_scaled = LinearRegression(fit_intercept=True)
reg_scaled.fit(X, Y_all)
print(reg_scaled.coef_)
reg_scaled.score(X, Y_all)

In [ ]:
reg_std = LinearRegression()
reg_std.fit(X_std, Y_all)
print(reg_std.coef_)
reg_std.score(X_std, Y_all)

In [ ]:
from sklearn.linear_model import Lasso, Ridge

def regularize_different_alpha(name: str, X, Y):
    print('-' * 50)
    print('feature_std=', X.std(axis=0))
    print('feature_mean=', X.mean(axis=0))
    alpha_list = 10.**np.arange(-7, 8)
    for alpha in alpha_list:
        if name.lower() == 'ridge':
            reg = Ridge(alpha=alpha, normalize=False)
        elif name.lower() == 'lasso':
            reg = Lasso(alpha=alpha, normalize=False)
        else:
            raise ValueError('Unrecongnized name')
        reg.fit(X, Y)
        print(f'alpha={alpha}, score={reg.score(X, Y):.4f}')
        print('coef=', reg.coef_[:8])  # only show first 8 coefs (total 10)


In [ ]:
regularize_different_alpha('ridge', X, Y_all)
regularize_different_alpha('ridge', X_std, Y_all)

In [ ]:
regularize_different_alpha('lasso', X, Y_all)
regularize_different_alpha('lasso', X_std, Y_all)